In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from typing import Tuple, List, Dict
import os
from pathlib import Path
from PIL import Image
from torchvision.transforms import Compose
from torch.utils.data import Dataset
import torch.optim as optim
import torch
import numpy as np
import torchvision.transforms
import torchvision.datasets as dts
from torchvision.transforms import ToTensor
import torch
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
# from torchmetrics import Accuracy

In [ ]:
!pip install -q keras

In [ ]:
from torchmetrics import Accuracy

In [ ]:
!pip install -q torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
import keras

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, root_dir: str, transform: Compose = None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.path = list(Path(self.root_dir).glob("**/*.jpg"))
        self.transform = transform
        self.classes, self.classes_to_idx = self.__find_classes()

    def __getitem__(self, index) -> Tuple[Image.Image, int]:
        image = self.__load_image(index)
        class_name = self.__get_class(index)
        class_idx = self.classes_to_idx[class_name]

        if self.transform:
            image = self.transform(image)

        return image, class_idx

    def __len__(self) -> int:
        return len(self.path)

    def __find_classes(self) -> Tuple[List[str], Dict]:
        classes = sorted(entry.name for entry in os.scandir(self.root_dir) if entry.is_dir())
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx
    def __load_image(self, index: int) -> Image.Image:
        return Image.open(self.path[index]).convert("RGB")

    def __get_class(self, index: int) -> str:
        return self.path[index].parent.name

In [ ]:
class binary_classification(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.block = nn.Sequential(
            nn.Linear(128*128*3, 100), #2?
            nn.ReLU(), #activation function (linearity)
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
        )

    def forward(self, X):
        X=X.view(-1,128*128*3)
        y=self.block(X)
        # print(y.shape)
        # X=X.squeeze()
        return y.squeeze(dim=1)
model=binary_classification()
image,labels=next(iter(train_loader))
with torch.inference_mode():

    prediction = model((image))
    # print(image)
    # print(image.shape)
    # print(type(image))
    # prediction = prediction.argmax(dim=1)

print(f"Prediction: {prediction}")

Prediction: tensor([-0.1581, -0.1696, -0.1751, -0.1719, -0.1818, -0.1827, -0.2174, -0.1882,
        -0.1600, -0.2108, -0.1521, -0.1485])


In [ ]:
def plot_stats(epochs,train_loss,test_loss):
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_loss, label='Train loss')
    plt.plot(epochs, test_loss, label='Test loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize,Resize
train_transforms = Compose([
    Resize((128, 128)),
    ToTensor(),
    # Normalize([0.5, 0.5, 0.5],
    #                         [0.5, 0.5, 0.5])
])

In [217]:
def train (model,train_loader,test_loader,optimizer,loss_function,epochs=100):
    epochs_count, train_loss_arr, test_loss_arr = [], [], []
    for epoch in range(epochs):
        model.train()
        print (epoch)

        for X, y in train_loader:
            # X=X.squeeze()
            output = model(X)
            train_loss = loss_function(output, y.float())
            train_loss.backward()
            # print("HERE")
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        for X ,y in test_loader:
            with torch.inference_mode():

             output= model(X)
             test_loss = loss_function(output,y)

        if epoch % 10 == 0:
                epochs_count.append(epoch)
                train_loss_arr.append(train_loss.detach().numpy())
                test_loss_arr.append(test_loss.detach().numpy())
                print(f"Epoch {epoch}: train loss {train_loss}, test loss {test_loss}")
    return epochs_count, train_loss_arr, test_loss_arr


def accuracy(model, data_loader):

        acc = Accuracy(task='multiclass', num_classes=2)
        model.eval()
        for X, y in data_loader:
                output = model(X)
                acc.update(output, y)
        return acc.compute()

In [ ]:
def predict(model, data):
    model.eval()
    with torch.inference_mode():

        y_preds = model(data)
        y_preds = torch.sigmoid(y_preds)
        y_preds = torch.round(y_preds)
        return y_preds

In [ ]:
def plot_stats(epochs,train_loss,test_loss):
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_loss, label='Train loss')
    plt.plot(epochs, test_loss, label='Test loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [218]:
train_cats_dogs = CatsDogsDataset(root_dir="/content/drive/MyDrive/PETS/train",transform=train_transforms)
test_cats_dogs = CatsDogsDataset(root_dir="/content/drive/MyDrive/PETS/test",transform=train_transforms)
print(len(train_cats_dogs))
print(type(train_cats_dogs))
train_loader = DataLoader(dataset=train_cats_dogs, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_cats_dogs, batch_size=1, shuffle=True)
# from torchvision.datasets import ImageFolder
# train_data = ImageFolder(root="/content/drive/MyDrive/TRY/train", transform=train_transforms)
# test_data = ImageFolder(root="/content/drive/MyDrive/TRY/test", transform=train_transforms)
# train_loader = DataLoader(dataset=train_data, batch_size=32, num_workers=2, shuffle=True)
# test_loader = DataLoader(dataset=test_data, batch_size=32, num_workers=2, shuffle=False)


15177
<class '__main__.CatsDogsDataset'>


In [219]:
model=binary_classification()
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01) #model?


In [1]:
epochs_count, train_losses, test_losses = train (model,train_loader,test_loader,optimizer,loss_function,epochs=100)
print(f"Train Accuracy: {accuracy(model, train_loader):.2f}")
print(f"Test Accuracy: {accuracy(model, test_loader):.2f}")

In [ ]:
plot_stats(epochs_count,train_losses,test_losses)

In [ ]:
import cv2
def display_random_image(dataset,classes: List[str] = None):
  random_number = random.randint(0,11 )
  Image,label=train_cats_dogs[random_number]
  # plt.imshow(imag.numpy().squeeze())


  prediction = predict(model, Image).item()
  color = 'green' if label == prediction else 'red'

  plt.imshow(Image.permute(1,2,0))
  plt.axis('off')
  plt.title(f"Class: {classes[label]} ")
  # plt.title(f'Actual: {label} | Prediction: {prediction}', color=color)
display_random_image(train_cats_dogs,train_cats_dogs.classes)